In [1]:
from __future__ import print_function 
import os, sys, h5py
import numpy as np
from six.moves import cPickle
from collections import OrderedDict

import tensorflow as tf
sys.path.append('..')
from tfomics import neuralnetwork as nn
from tfomics import utils, learn

# import models
from model_zoo import fourthplace_connectomics_model
from model_zoo import simple_connectomics_model, simple_connectomics_model2
from model_zoo import residual_connectomics_model_PK, residual_connectomics_model, residual_connectomics_model2

# load data

In [2]:
filename = 'processed_dataset.hdf5'
data_path = 'D:/Dropbox/TFconnect'
filepath = os.path.join(data_path,filename)

group_name = ['processed_data']
dataset = h5py.File(filepath,'r')
%time dtf = np.array(dataset['/'+group_name[0]+'/dtf'])
ltf = np.array(dataset['/'+group_name[0]+'/ltf'])
dtf_crossval = np.array(dataset['/'+group_name[0]+'/dtf_crossval'])
ltf_crossval = np.array(dataset['/'+group_name[0]+'/ltf_crossval'])

X_train = dtf#[:10000,:,:,:]
y_train = ltf#[:10000,:]
X_valid = dtf_crossval#[:5000,:,:,:]
y_valid = ltf_crossval#[:5000,:]

Wall time: 2.64 s


# fourth place competition

In [ ]:
# get shapes
num_data, height, width, dim = X_train.shape
input_shape=[None, height, width, dim]
num_labels = y_train.shape[1]  

# load model
net, placeholders, optimization = fourthplace_connectomics_model.model(input_shape, num_labels)

# build neural network class
nnmodel = nn.NeuralNet(net, placeholders)
nnmodel.inspect_layers()

# set output file paths
resultspath = utils.make_directory('../results', 'tfomics')
output_name = '1d_version'
filepath = os.path.join(resultspath, output_name)

# compile neural trainer
nntrainer = nn.NeuralTrainer(nnmodel, optimization, save='best', filepath=filepath)

In [ ]:
X = {'inputs': X_train, 'targets': y_train, 'keep_prob': 0.8, 'is_training': True}
X2 = {'inputs': X_valid, 'targets': y_valid, 'keep_prob': 1, 'is_training': False}
data = {'train': X, 'valid': X2}
learn.train_minibatch(nntrainer, data, batch_size=1000, num_epochs=50, 
                    patience=5, verbose=2, shuffle=True)

In [ ]:
nntrainer.close_sess()

# Simple connectomics model

In [ ]:
optimization

In [ ]:
# get shapes
num_data, height, width, dim = X_train.shape
input_shape=[None, height, width, dim]
num_labels = y_train.shape[1]  

# load model
net, placeholders, optimization = simple_connectomics_model.model(input_shape, num_labels)

# build neural network class
nnmodel = nn.NeuralNet(net, placeholders)
nnmodel.inspect_layers()

# set output file paths
resultspath = utils.make_directory('../results', 'tfomics')
output_name = '1d_version'
filepath = os.path.join(resultspath, output_name)

# compile neural trainer
nntrainer = nn.NeuralTrainer(nnmodel, optimization, save='best', filepath=filepath)

In [ ]:
train = {'inputs': X_train, 'targets': y_train, 'keep_prob': 0.8, 'is_training': True}
valid = {'inputs': X_valid, 'targets': y_valid, 'keep_prob': 1, 'is_training': False}
data = {'train': train, 'valid': valid}
learn.train_minibatch(nntrainer, data, batch_size=1000, num_epochs=50, 
                    patience=5, verbose=2, shuffle=True)

In [ ]:
nntrainer.close_sess()

# Simple connectomics model

In [ ]:
# get shapes
num_data, height, width, dim = X_train.shape
input_shape=[None, height, width, dim]
num_labels = y_train.shape[1]  

# load model
net, placeholders, optimization = simple_connectomics_model2.model(input_shape, num_labels)

# build neural network class
nnmodel = nn.NeuralNet(net, placeholders)
nnmodel.inspect_layers()

# set output file paths
resultspath = utils.make_directory('../results', 'tf-test')
output_name = '1d_version2'
filepath = os.path.join(resultspath, output_name)

# compile neural trainer
nntrainer = nn.NeuralTrainer(nnmodel, optimization, save='best', filepath=filepath)

In [ ]:
X = {'inputs': X_train, 'targets': y_train, 'keep_prob': 0.8, 'is_training': True}
X2 = {'inputs': X_valid, 'targets': y_valid, 'keep_prob': 1, 'is_training': False}
data = {'train': X, 'valid': X2}
learn.train_minibatch(nntrainer, data, batch_size=100, num_epochs=50, 
                    patience=5, verbose=2, shuffle=True)

In [ ]:
nntrainer.close_sess()

# residual model

In [3]:
# get shapes
num_data, height, width, dim = X_train.shape
input_shape=[None, height, width, dim]
num_labels = y_train.shape[1]  

# load model
net, placeholders, optimization = residual_connectomics_model_PK.model(input_shape, num_labels)

# build neural network class
nnmodel = nn.NeuralNet(net, placeholders)

# set output file paths
resultspath = utils.make_directory('../results', 'tfomics')
output_name = 'residual'
filepath = os.path.join(resultspath, output_name)

# compile neural trainer
nntrainer = nn.NeuralTrainer(nnmodel, optimization, save='best', filepath=filepath)

In [4]:
nntrainer.set_best_parameters()

loading model from:  ../results\tfomics\residual_best.ckpt


In [ ]:
train = {'inputs': X_train, 'targets': y_train, 'keep_prob': 0.8, 'is_training': True}
valid = {'inputs': X_valid, 'targets': y_valid, 'keep_prob': 1, 'is_training': False}
data = {'train': train, 'valid': valid}
learn.train_minibatch(nntrainer, data, batch_size=100, num_epochs=50, 
                    patience=5, verbose=2, shuffle=True)

In [5]:
test = {'inputs': X_valid, 'targets': y_valid, 'keep_prob': 1, 'is_training': False}
test_loss = nntrainer.test_model(test)

  test  loss:		0.34043
  test  accuracy:	0.86718+/-0.00000
  test  auc-roc:	0.93357+/-0.00000
  test  auc-pr:		0.93072+/-0.00018


In [ ]:
dataset

In [6]:
filename = 'D:/Dropbox/TFconnect/valideval_dataset.hdf5'
dataset = h5py.File(filename,'r')
group_name = ['valid_data']
%time val_dat = np.array(dataset['/'+group_name[0]+'/vs_valid'])
val_lbl = np.array(dataset['/'+group_name[0]+'/label_valid'])

Wall time: 5 ms


In [7]:
fragLen = 330
N = 14
avg_F = np.mean(val_dat,axis=0)

startgap = np.ceil((val_dat.shape[1] - fragLen)/N).astype('int')
true_lbl = np.zeros((val_dat.shape[0]*val_dat.shape[0],), dtype='float32')
pred_lbl = np.zeros((val_dat.shape[0]*val_dat.shape[0],), dtype='float32')

# Counter for the "true_lbl" array
cnt_ = 0
# Counter for the "pred_lbl" array
cnt_u = 0
for a in range(val_dat.shape[0]):
    if a%100 == 0:
        print('\r' + 'X'*(a//100))

    # Create batch array to send thru network
    im_eval = np.empty((N*val_dat.shape[0],3,fragLen,1), dtype='float32')

    # Count the number of traces in each batch
    cnt = 0

    for b in range(val_dat.shape[0]):

        for n in range(0, val_dat.shape[1] - fragLen, startgap):
            try:
                im_eval[cnt,:,:,0] = np.vstack((val_dat[a,n:n+fragLen],
                                     val_dat[b,n:n+fragLen],
                                     avg_F[n:n+fragLen]))
            except:
                from IPython.core.debugger import Tracer
                Tracer()()

            cnt += 1

        # Keep track of the true labels
        if val_lbl[a,b] == 1:
            true_lbl[cnt_] = 1
        else:
            true_lbl[cnt_] = 0

        cnt_ += 1

    # Run batch through network
    test = {'inputs': im_eval, 'keep_prob': 1, 'is_training': False}
    pred_stop = nntrainer.get_activations(test, layer='output')[:,0]
    # Average output over each group of N traces
    for u in range(0, len(pred_stop), N):
        pred_lbl[cnt_u] = np.mean(pred_stop[u:u+N])
        cnt_u += 1        


X
XX
XXX
XXXX
XXXXX
XXXXXX
XXXXXXX
XXXXXXXX
XXXXXXXXX


In [9]:
from sklearn.metrics import roc_curve, auc, precision_recall_curve, accuracy_score, roc_auc_score
fpr, tpr, thresholds = roc_curve(true_lbl, pred_lbl)
wrk = auc(fpr, tpr)
print(wrk)

0.935433871545


In [ ]:
X_test = dtf[0:1,:,:,:]
test = {'inputs': X_test, 'keep_prob': 1, 'is_training': False}
predictions = nntrainer.get_activations(test, layer='output')

In [ ]:
im_eval.shape

In [ ]:
pred_stop = nntrainer.get_activations(im_eval[0:1,:,:,:], layer='output')

In [ ]:
nntrainer.close_sess()

# residual model 2

In [ ]:
# get shapes
num_data, height, width, dim = X_train.shape
input_shape=[None, height, width, dim]
num_labels = y_train.shape[1]  

# load model
net, placeholders, optimization = residual_connectomics_model2.model(input_shape, num_labels)

# build neural network class
nnmodel = nn.NeuralNet(net, placeholders)

# set output file paths
resultspath = utils.make_directory('../results', 'tf-test')
output_name = '1d_version_residual2'
filepath = os.path.join(resultspath, output_name)

# compile neural trainer
nntrainer = nn.NeuralTrainer(nnmodel, optimization, save='best', filepath=filepath)

In [ ]:
train = {'inputs': X_train, 'targets': y_train, 'keep_prob': 0.8, 'is_training': True}
valid = {'inputs': X_valid, 'targets': y_valid, 'keep_prob': 1, 'is_training': False}
data = {'train': train, 'valid': valid}
learn.train_minibatch(nntrainer, data, batch_size=1000, num_epochs=50, 
                    patience=5, verbose=2, shuffle=True)

In [ ]:
nntrainer.close_sess()